In [1]:
from source.inputters.corpus import KnowledgeCorpus
corpus = KnowledgeCorpus('./data/', 'demo', max_len=500, max_vocab_size=30000, share_vocab=True)
corpus.load()
valid_raw = corpus.read_data('./data/demo.dev', data_type='valid')
valid_data = corpus.build_examples(valid_raw)

from source.inputters.dataset import Dataset
data = Dataset(valid_data)
loader = data.create_batches(5)
for input in loader:
    input = input
    break

Loading prepared vocab from ./data/demo_30000.vocab.pt ...
Vocabulary size of fields: SRC-30004 TGT-30004 CUE-30004
Loading prepared data from ./data/demo_30000.data.pt ...
Number of examples: TRAIN-89901 VALID-9054 TEST-9054
Read 9054 VALID examples (0 filtered)


100%|████████████████████████████████████████████████████████████████████████████| 9054/9054 [00:01<00:00, 7941.35it/s]


In [2]:
enc_inputs = input
# y(t-1)
dec_inputs = input.tgt[0][:, :-1], input.tgt[1] - 1
# y(t)
target = input.tgt[0][:, 1:]

In [3]:
enc_inputs.src

(tensor([[   2,   30,    4,    8,    8,   36,    4,   29,   96,    3,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0],
         [   2,   30,    4,    8,    8,   36,    4,   29,   45,   26,   50,   11,
            37,  292,   45,   26,   50,   11,   10,    3,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0],
         [   2,   30,    4,    8,    8,   36,    4,   29,   45

In [4]:
from source.models.pgnet import PointerNet
model = PointerNet(corpus.SRC.vocab_size, 300, 800, padding_idx=corpus.padding_idx)

In [5]:
outputs, dec_init_state = model.encode(enc_inputs, None)

In [6]:
dec_init_state.fact_mask[0]

tensor([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       dtype=torch.uint8)

In [7]:
dec_init_state.fact[0]

tensor([    4,    12,    11,     3,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     4,    22,
        26107,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     4,    17,    28,    21,
          161,  7378,   355, 11000,     6,   153,  2381,   752,    39,     3,
            0,     0,     0,     0,     4,   104,   205,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     8,    34,     1,   418,     7,  1252,     1,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            8,    36,  7961,     3,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     8,   182,
          179,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     8,    35, 

In [8]:
import torch

In [9]:
dec_inputs, _ = dec_inputs
input = dec_inputs[:, 0]

In [10]:
prob_vocab, p_mode, attn_f, attn_h, fact, hist = model.decoder.decode(input, dec_init_state)

In [11]:
weighted_prob = prob_vocab * p_mode[:, :, 0].unsqueeze(1)
weighted_f = attn_f * p_mode[:, :, 1].unsqueeze(1)
weighted_h = attn_h * p_mode[:, :, 2].unsqueeze(1)

In [12]:
from source.utils.misc import convert_dist

In [13]:
weighted_prob = convert_dist(
                weighted_h, hist, weighted_prob)

In [14]:
weighted_prob = convert_dist(
                weighted_f, fact, weighted_prob)

In [15]:
attn_f[0]

tensor([[0.0144, 0.0148, 0.0143, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0137, 0.0150, 0.0139, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0145, 0.0139, 0.0133, 0.0119, 0.0141, 0.0138, 0.0145, 0.0168, 0.0157,
         0.0137, 0.0134, 0.0155, 0.0136, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0147, 0.0162, 0.0146, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0127, 0.0149, 0.0148, 0.0152, 0.0148, 0.0152, 0.0148, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0130, 0.0157, 0.0180, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0123, 0.0129, 0.0

In [16]:
# out_facts, fact, hist, out_hists, prob_vocab, prob_hist, prob_fact, p_modes = model(enc_inputs, dec_inputs)

In [17]:
# model(enc_inputs, dec_inputs)

In [1]:
from source.inputters.corpus2 import HieraSrcCorpus
corpus = HieraSrcCorpus('./data/', 'hsrc', max_len=500, max_vocab_size=30000, share_vocab=True)
corpus.load()

valid_raw = corpus.read_data('./data/hsrc.dev', data_type='valid')
valid_data = corpus.build_examples(valid_raw)

from source.inputters.dataset import Dataset
data = Dataset(valid_data)
loader = data.create_batches(5)
for input in loader:
    input = input
    break

Loading prepared vocab from ./data/hsrc_30000.vocab.pt ...
Vocabulary size of fields: SRC-30004 TGT-30004 CUE-30004
Loading prepared data from ./data/hsrc_30000.data.pt ...
Number of examples: TRAIN-89901 VALID-9054 TEST-9054


100%|████████████████████████████████████████████████████████████████████████████| 9054/9054 [00:01<00:00, 8838.20it/s]


In [2]:
input.src

(tensor([[[   2,   29,    4,    8,    8,   35,    4,    3,    0,    0,    0,
              0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [   2,   95,    3,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
              0,    0,    0],
          [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
   

In [3]:
from source.modules.embedder import Embedder
from source.modules.encoders.rnn_encoder import RNNEncoder
from source.modules.encoders.rnn_encoder import HRNNEncoder

In [4]:
embedder = Embedder(num_embeddings=corpus.SRC.vocab_size,
                                embedding_dim=300,
                                padding_idx=corpus.padding_idx)
sub_encoder = RNNEncoder(input_size=300,
                          hidden_size=800,
                          embedder=embedder,
                          num_layers=1,
                          bidirectional=False,
                          dropout=0.0)
hiera_encoder = RNNEncoder(input_size=800,
                          hidden_size=800,
                          num_layers=1,
                          bidirectional=False,
                          dropout=0.0)

In [5]:
encoder = HRNNEncoder(sub_encoder, hiera_encoder)

In [6]:
indices, lengths = input.src
batch_size, max_hiera_len, max_sub_len = indices.size()
hiera_lengths = lengths.gt(0).long().sum(dim=1)

In [7]:
hiera_lengths

tensor([2, 3, 5, 7, 2])

In [10]:
hiera_outputs, hiera_hidden, _ = encoder(input.src)

In [11]:
hiera_outputs.size()

torch.Size([5, 7, 800])

In [12]:
hiera_hidden.size()

torch.Size([1, 5, 800])

In [2]:
from source.models.hseq2seq import HSeq2Seq
model = HSeq2Seq(corpus.SRC.vocab_size, corpus.SRC.vocab_size, 300, 800, padding_idx=corpus.padding_idx)

In [3]:
model.encode(input)

({}, <source.modules.decoders.state.DecoderState at 0x26bb5007c50>)